# Forecasting Future Market Trends: TSLA

This notebook uses trained ARIMA and LSTM models to forecast Tesla's future stock prices over a 6-12 month horizon, visualize results, and provide risk/trend analysis.

In [ ]:
# Install dependencies if needed (uncomment if running locally)
# !pip install yfinance pandas numpy matplotlib seaborn statsmodels tensorflow scikit-learn pmdarima

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.arima.model import ARIMA
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import pmdarima as pm
sns.set(style='whitegrid')
%matplotlib inline

## 1. Load Data and Refit Models if Needed
(Or load models from Task 2 if saved)

In [ ]:
# Download latest TSLA close prices
tsla = yf.download('TSLA', start='2015-01-01', auto_adjust=True)['Close']
# Use all available data for model refit
train = tsla
forecast_period = 126  # ~6 months (126 trading days)
forecast_period_12m = 252  # ~12 months


## 2. ARIMA Forecast (with Confidence Intervals)

In [ ]:
# Use auto_arima for best (p,d,q)
arima_model = pm.auto_arima(train, seasonal=False, stepwise=True, suppress_warnings=True, trace=True)
arima_order = arima_model.order
arima_fit = ARIMA(train, order=arima_order).fit()
arima_forecast_6m = arima_fit.get_forecast(steps=forecast_period)
arima_pred_6m = arima_forecast_6m.predicted_mean
arima_ci_6m = arima_forecast_6m.conf_int()
arima_forecast_12m = arima_fit.get_forecast(steps=forecast_period_12m)
arima_pred_12m = arima_forecast_12m.predicted_mean
arima_ci_12m = arima_forecast_12m.conf_int()


In [ ]:
plt.figure(figsize=(14,7))
plt.plot(train, label='Historical')
plt.plot(arima_pred_6m.index, arima_pred_6m, label='ARIMA Forecast (6M)')
plt.fill_between(arima_ci_6m.index, arima_ci_6m.iloc[:,0], arima_ci_6m.iloc[:,1], color='gray', alpha=0.3, label='Confidence Interval (6M)')
plt.title('TSLA ARIMA 6-Month Forecast with Confidence Interval')
plt.legend()
plt.show()

plt.figure(figsize=(14,7))
plt.plot(train, label='Historical')
plt.plot(arima_pred_12m.index, arima_pred_12m, label='ARIMA Forecast (12M)')
plt.fill_between(arima_ci_12m.index, arima_ci_12m.iloc[:,0], arima_ci_12m.iloc[:,1], color='gray', alpha=0.3, label='Confidence Interval (12M)')
plt.title('TSLA ARIMA 12-Month Forecast with Confidence Interval')
plt.legend()
plt.show()

## 3. LSTM Forecast (Point Estimate Only)
*Note: LSTM does not natively provide confidence intervals. Bootstrapping or MC Dropout can be used for uncertainty, but are not included here for brevity.*

In [ ]:
lookback = 30
scaler = MinMaxScaler()
scaled = scaler.fit_transform(train.values.reshape(-1,1))
def make_lstm_sequences(data, lookback):
    X = []
    for i in range(lookback, len(data)):
        X.append(data[i-lookback:i, 0])
    return np.array(X)
X_train = make_lstm_sequences(scaled, lookback)
y_train = scaled[lookback:]
model = Sequential([
    LSTM(50, activation='relu', input_shape=(lookback,1)),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=1)
def lstm_forecast_point(model, last_seq, n_steps):
    preds = []
    seq = last_seq.copy()
    for _ in range(n_steps):
        pred = model.predict(seq.reshape(1, lookback, 1), verbose=0)[0,0]
        preds.append(pred)
        seq = np.roll(seq, -1)
        seq[-1] = pred
    return scaler.inverse_transform(np.array(preds).reshape(-1,1)).flatten()
last_seq = scaled[-lookback:]
lstm_pred_6m = lstm_forecast_point(model, last_seq, forecast_period)
lstm_pred_12m = lstm_forecast_point(model, last_seq, forecast_period_12m)


In [ ]:
plt.figure(figsize=(14,7))
plt.plot(train, label='Historical')
plt.plot(pd.date_range(train.index[-1], periods=forecast_period+1, freq='B')[1:], lstm_pred_6m, label='LSTM Forecast (6M)')
plt.title('TSLA LSTM 6-Month Forecast')
plt.legend()
plt.show()

plt.figure(figsize=(14,7))
plt.plot(train, label='Historical')
plt.plot(pd.date_range(train.index[-1], periods=forecast_period_12m+1, freq='B')[1:], lstm_pred_12m, label='LSTM Forecast (12M)')
plt.title('TSLA LSTM 12-Month Forecast')
plt.legend()
plt.show()

## 4. Trend, Volatility, and Risk Analysis

- **Trend Analysis:**
    - ARIMA and LSTM both provide point forecasts. ARIMA's confidence intervals widen over time, indicating increasing uncertainty.
    - Look for upward, downward, or stable trends in the forecast.
    - Note any anomalies or sudden changes.
- **Volatility and Risk:**
    - The width of ARIMA's confidence intervals reflects forecast uncertainty.
    - Wider intervals at longer horizons mean less reliability in long-term forecasts.
- **Market Opportunities and Risks:**
    - If the forecast shows an upward trend, there may be investment opportunities, but high volatility or wide confidence intervals increase risk.
    - If the forecast is downward or highly volatile, caution is warranted.

*Always combine model-based insights with fundamental analysis and risk management.*